In [3]:

import nltk,re,time,umap,collections,os,string,sys,mlflow
sys.path.append(os.path.abspath(".."))
from modules import utils,modeling

from sentence_transformers import SentenceTransformer,util
import plotly.express as px
import pandas as pd
import glob

from sklearn.metrics import completeness_score,homogeneity_score, v_measure_score,normalized_mutual_info_score,adjusted_mutual_info_score,silhouette_score
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from summa.summarizer import summarize
from summa.keywords import keywords
from hdbscan import HDBSCAN
from sentence_transformers import models
from sklearn.preprocessing import normalize
from torch import nn
import mlflow.sklearn
from  mlflow.tracking import MlflowClient
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_distances

import seaborn as sns
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from textwrap import wrap
stopwords = utils.load_stopwords()
from flexible_clustering import FISHDBC
import flexible_clustering

remote_server="http://ec2-18-193-70-141.eu-central-1.compute.amazonaws.com"
client = MlflowClient(remote_server)
mlflow.set_tracking_uri(remote_server)


%matplotlib inline
%load_ext autoreload
%autoreload 2

%reload_ext utils
%reload_ext modeling
os.environ["MLFLOW_S3_ENDPOINT_URL"]="http://s3.eu-central-1.amazonaws.com"
os.environ["MLFLOW_TRACKING_USERNAME"]="mlflow"
os.environ["MLFLOW_TRACKING_PASSWORD"]="mlflow"

FISH = FISHDBC(cosine_distances)

Number of stopwords 607
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
X = utils.load_labeled_data()
X.drop(["site","entities_analyzed","updated_at"],inplace=True,axis="columns")

labeled = pd.read_csv("../data/raw_article_entities.csv",header=None,names=["_id","article_id",
                                                                            "kicker_headline_NER","text_NER",
                                                                            "seo_title_NER","created_at_ne",
                                                                            "updated_at","parset"])

labeled.drop(["parset","updated_at","_id"],axis=1,inplace=True)
labeled = labeled.set_index("article_id")


X['mix'] =  X['headline'] +" "+ X["text"]
#pdf_mix = utils.preprocess_text(X,col="mix")
pdf_mix = X["mix"].values
X = X.join(labeled,how='right')
X.drop_duplicates(subset='headline',inplace=True)
pdf= utils.preprocess_articles_for_bert(X,"mix")
y = X.labels
X.drop("labels",axis="columns",inplace=True)
X.tail(1)

,headline,seo_title,text,created_at_date,created_at,mix,kicker_headline_NER,text_NER,seo_title_NER,created_at_ne
fde4b3c95901643bf957060e97458af3c3a5cb77979d6b1cab666bd11a343918,„FameMaker“: Wie Stefan Raab die Fäden weiter ...,Wie Raab hinter der Kamera die Fäden zieht,Bühne frei für ein neues bisschen Raab ... Am ...,9/14/20,2020-09-14 18:27:15,„FameMaker“: Wie Stefan Raab die Fäden weiter ...,"[{""entities"":[{""mentions"":[{""text"":{""content"":...","[{""entities"":[{""mentions"":[{""text"":{""content"":...","[{""entities"":[{""mentions"":[{""text"":{""content"":...",2020-09-14 18:28:06


In [5]:
model = models.Transformer( 'T-Systems-onsite/bert-german-dbmdz-uncased-sentence-stsb',max_seq_length=225)
embeddings = modeling.get_sentence_embeddings(pdf,model,False)
#embeddings[0]


--- Embedding dimension 768
--- 239 Documnets encoded 43.29958915710449 seconds ---


In [6]:
mlflow.set_experiment("basic-eval-FISH")


In [ ]:
y_summarized_clusters = dict(collections.Counter(y.values))

params={"min_samples":2,"ef":200}

efs= [100,200,300,350,400,500,600,700]
selection_methods= ['leaf','eom']
for ef in efs:
    for selection_method in selection_methods:
        params['ef']=ef
        with mlflow.start_run():
            mlflow.log_params(params)
        
            clusterer = flexible_clustering.FISHDBC(cosine_distances,**params)
            for elem in embeddings:
                clusterer.add([elem])
            labels, probs, stabilities, condensed_tree, slt, mst = clusterer.cluster(cluster_selection_method=selection_method,min_cluster_size=3)
            mlflow.log_param("cluster_selection_method",selection_method)
            mlflow.log_metric(key="completeness_score", value=completeness_score(labels,y.values))
            mlflow.log_metric(key="v_measure_score", value=v_measure_score(labels,y.values))
            mlflow.log_metric(key="homogeneity_score", value=homogeneity_score(labels,y.values))
            mlflow.log_metric(key="normalized_mutual_info_score", value=v_measure_score(labels,y.values))
            
            summarized_clusters = dict(collections.Counter(labels))
            try:
                mlflow.log_metric(key="outliers_ratio", value=(summarized_clusters[-1]/len(labels)))
            except Exception as err:
                mlflow.log_metric(key="outliers_ratio", value=0)
        
            mlflow.log_metric(key="unique_cluters", value=len(summarized_clusters.items()))
            mlflow.log_metric(key="clusters_ratio_to_GT", value=(len(summarized_clusters.items())-1)/len(y_summarized_clusters.items()))
            mlflow.log_metric(key="silhuette_score", value=silhouette_score(embeddings,labels))
            mlflow.log_metric(key="silhuette_score_true", value=silhouette_score(embeddings,y.values))
            results = utils.link_to_raw_data(results,X,labels)
    
            try:
                modeling.scatter_plot(results,save_fig=True)
                mlflow.log_artifact("./tmp_scatter_plot.html")
            except Exception as err:
                print(err)
            print("Done!")